# Deep Learning Grid Search

In this project, we will learn how to use the scikit-learn grid search capability.

We are going to learn the following topics:

* How to use Keras models in scikit-learn.
* How to use grid search in scikit-learn.
* How to tune batch size and training epochs.
* How to tune learning rate
* How to tune network weight initialization.
* How to tune activation functions.
* How to tune dropout regularization.
* How to tune the number of neurons in the hidden layer.

In [1]:
import sys
import pandas
import numpy
import sklearn
import keras

print('Python: {}'.format(sys.version))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))
print('Sklearn: {}'.format(sklearn.__version__))
print('Keras: {}'.format(keras.__version__))

Using TensorFlow backend.


Python: 3.7.3 (default, Apr 24 2019, 15:29:51) [MSC v.1915 64 bit (AMD64)]
Pandas: 0.24.2
Numpy: 1.16.4
Sklearn: 0.21.2
Keras: 2.3.1


In [2]:
import pandas as pd
import numpy as np

# import the uci pima indians diabetes dataset
url = "diabetes.csv"
names = ['n_pregnant', 'glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)',
        'BMI', 'pedigree_function', 'age', 'class']
df = pd.read_csv(url)

#Change the columns name
df.columns = names

In [3]:
# Describe the dataset
df.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [4]:
df[df['glucose_concentration'] == 0]

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
75,1,0,48,20,0,24.7,0.140,22,0
182,1,0,74,20,23,27.7,0.299,21,0
342,1,0,68,35,0,32.0,0.389,22,0
349,5,0,80,32,0,41.0,0.346,37,1
502,6,0,68,41,0,39.0,0.727,41,1


In [5]:
# Preprocess the data, mark zero values as NaN and drop
columns = ['glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)', 'BMI']

for col in columns:
    df[col].replace(0, np.NaN, inplace=True)
    
df.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
# Drop rows with missing values
df.dropna(inplace=True)

# summarize the number of rows and columns in df
df.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
# Convert dataframe to numpy array
dataset = df.values
print(dataset.shape)

(392, 9)


In [8]:
# split into input (X) and an output (Y)
X = dataset[:,0:8]
Y = dataset[:, 8].astype(int)

In [9]:
print(X.shape)
print(Y.shape)
print(Y[:5])

(392, 8)
(392,)
[0 1 1 1 1]


In [10]:
# Normalize the data using sklearn StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)

In [11]:
print(scaler)

StandardScaler(copy=True, with_mean=True, with_std=True)


In [12]:
# Transform and display the training data
X_standardized = scaler.transform(X)

data = pd.DataFrame(X_standardized)
X_standardized 

array([[-0.7174265 , -1.09104581, -0.37365481, ..., -0.710421  ,
        -1.03187632, -0.9682991 ],
       [-1.02921274,  0.46631407, -2.45696436, ...,  1.42673006,
         5.11511079,  0.2095853 ],
       [-0.09385402, -1.44794079, -1.65569146, ..., -0.29723846,
        -0.79712575, -0.47751393],
       ...,
       [-0.40564026, -1.12349081, -1.01467313, ..., -0.66767798,
         0.70411863, -0.87014206],
       [ 2.08864966, -0.70170584,  0.42761809, ..., -0.02653266,
        -1.0202837 ,  3.15429628],
       [ 0.52971846, -0.05280589,  0.10710893, ..., -0.9811268 ,
        -0.80582021, -0.0848858 ]])

In [13]:
# import necessary sklearn and keras packages
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

In [14]:
# Start defining the model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(40, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(20, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

model = create_model()
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 40)                360       
_________________________________________________________________
dense_2 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 1,201
Trainable params: 1,201
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
# Do a grid search for the optimal batch size and number of epochs
# import necessary packages
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(40, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(20, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, verbose = 1)

# define the grid search parameters
batch_size = [10, 20, 40]
epochs = [10, 50, 100]

# make a dictionary of the grid search parameters
param_grid = dict(batch_size=batch_size, epochs=epochs)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] batch_size=10, epochs=10 ........................................


C:\Users\Ajit kumar Singh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/10
261/261 [==============================] - 0s 712us/step - loss: 0.5018 - accuracy: 0.7739
Epoch 2/10
261/261 [==============================] - 0s 208us/step - loss: 0.4162 - accuracy: 0.8161
Epoch 3/10
261/261 [==============================] - 0s 286us/step - loss: 0.3901 - accuracy: 0.8276
Epoch 4/10
261/261 [==============================] - 0s 186us/step - loss: 0.3617 - accuracy: 0.8429
Epoch 5/10
261/261 [==============================] - 0s 242us/step - loss: 0.3460 - accuracy: 0.8506
Epoch 6/10
261/261 [==============================] - 0s 198us/step - loss: 0.3325 - accuracy: 0.8506
Epoch 7/10
261/261 [==============================] - 0s 192us/step - loss: 0.3267 - accuracy: 0.8506
Epoch 8/10
261/261 [==============================] - 0s 172us/step - loss: 0.3206 - accuracy: 0.8659
Epoch 9/10
261/261 [==============================] - 0s 185us/step - loss: 0.3155 - accuracy: 0.8582
Epoch 10/10
131/131 [==============================] - 0s 203us/step
[CV] ........

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s


Epoch 1/10
261/261 [==============================] - 0s 638us/step - loss: 0.5476 - accuracy: 0.7510
Epoch 2/10
261/261 [==============================] - 0s 245us/step - loss: 0.4509 - accuracy: 0.7931
Epoch 3/10
261/261 [==============================] - 0s 200us/step - loss: 0.4430 - accuracy: 0.8046
Epoch 4/10
261/261 [==============================] - 0s 199us/step - loss: 0.4145 - accuracy: 0.8199
Epoch 5/10
261/261 [==============================] - 0s 203us/step - loss: 0.4006 - accuracy: 0.8199
Epoch 6/10
261/261 [==============================] - 0s 310us/step - loss: 0.4110 - accuracy: 0.8199
Epoch 7/10
261/261 [==============================] - 0s 287us/step - loss: 0.4066 - accuracy: 0.8046
Epoch 8/10
261/261 [==============================] - 0s 275us/step - loss: 0.3951 - accuracy: 0.8276
Epoch 9/10
261/261 [==============================] - 0s 355us/step - loss: 0.3635 - accuracy: 0.8506
Epoch 10/10
131/131 [==============================] - 0s 761us/step
[CV] ........

...

Epoch 1/100
262/262 [==============================] - 0s 744us/step - loss: 0.5821 - accuracy: 0.6298
Epoch 2/100
262/262 [==============================] - 0s 299us/step - loss: 0.5177 - accuracy: 0.6870
Epoch 3/100
262/262 [==============================] - 0s 239us/step - loss: 0.5034 - accuracy: 0.7710
Epoch 4/100
262/262 [==============================] - 0s 316us/step - loss: 0.4901 - accuracy: 0.7634
Epoch 5/100
262/262 [==============================] - 0s 266us/step - loss: 0.4856 - accuracy: 0.7824
Epoch 6/100
262/262 [==============================] - 0s 264us/step - loss: 0.4704 - accuracy: 0.7748
Epoch 7/100
262/262 [==============================] - 0s 212us/step - loss: 0.4686 - accuracy: 0.7786
Epoch 8/100
262/262 [==============================] - 0s 221us/step - loss: 0.4585 - accuracy: 0.7901
Epoch 9/100
262/262 [==============================] - 0s 329us/step - loss: 0.4449 - accuracy: 0.8053
Epoch 10/100
262/262 [==============================] - 0s 287us/step - l

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   36.8s remaining:    0.0s


Epoch 1/10
261/261 [==============================] - 0s 583us/step - loss: 0.6018 - accuracy: 0.7126
Epoch 2/10
261/261 [==============================] - 0s 115us/step - loss: 0.4251 - accuracy: 0.7816
Epoch 3/10
261/261 [==============================] - 0s 118us/step - loss: 0.3890 - accuracy: 0.8046
Epoch 4/10
261/261 [==============================] - 0s 104us/step - loss: 0.3754 - accuracy: 0.8199
Epoch 5/10
261/261 [==============================] - 0s 111us/step - loss: 0.3614 - accuracy: 0.8544
Epoch 6/10
261/261 [==============================] - 0s 115us/step - loss: 0.3557 - accuracy: 0.8429
Epoch 7/10
261/261 [==============================] - 0s 120us/step - loss: 0.3490 - accuracy: 0.8544
Epoch 8/10
261/261 [==============================] - 0s 107us/step - loss: 0.3404 - accuracy: 0.8544
Epoch 9/10
261/261 [==============================] - 0s 107us/step - loss: 0.3309 - accuracy: 0.8467
Epoch 10/10
131/131 [==============================] - 0s 183us/step
[CV] ........

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  1.3min finished


Epoch 1/10
392/392 [==============================] - 0s 460us/step - loss: 0.5496 - accuracy: 0.7321
Epoch 2/10
392/392 [==============================] - 0s 140us/step - loss: 0.4573 - accuracy: 0.7704
Epoch 3/10
392/392 [==============================] - 0s 131us/step - loss: 0.4441 - accuracy: 0.7806
Epoch 4/10
392/392 [==============================] - 0s 146us/step - loss: 0.4208 - accuracy: 0.8163
Epoch 5/10
392/392 [==============================] - 0s 146us/step - loss: 0.4281 - accuracy: 0.8189
Epoch 6/10
392/392 [==============================] - 0s 152us/step - loss: 0.4045 - accuracy: 0.8163
Epoch 7/10
392/392 [==============================] - 0s 166us/step - loss: 0.4326 - accuracy: 0.8138
Epoch 8/10
392/392 [==============================] - 0s 149us/step - loss: 0.4061 - accuracy: 0.8138
Epoch 9/10
392/392 [==============================] - 0s 132us/step - loss: 0.3883 - accuracy: 0.8240
Epoch 10/10
392/392 [==============================] - 0s 150us/step - loss: 0.397

In [17]:
# Do a grid search for learning rate and dropout rate
# import necessary packages
from keras.layers import Dropout

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(learn_rate, dropout_rate):
    # create model
    model = Sequential()
    model.add(Dense(40, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(20, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

# make a dictionary of the grid search parameters
param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.733, total=   2.8s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s


[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.771, total=   3.1s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


...

[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.808, total=   3.4s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   26.2s remaining:    0.0s


[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.748, total=   3.4s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.756, total=   3.1s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.838, total=   2.9s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.1, learn_rate=0.01, score=0.748, total=   3.2s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.1, learn_rate=0.01, score=0.763, total=   3.2s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.1, learn_rate=0.01, score=0.831, total=   3.2s
[CV] dropout_rate=0.1, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.1, learn_rate=0.1, score=0.603, total=   4.8s
[CV] dropout_rate=0.1, learn_rate=0.1 ................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  1.5min finished


Best: 0.7882652916774457, using {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.7704081536859883 (0.030547750137858477) with: {'dropout_rate': 0.0, 'learn_rate': 0.001}
0.7704081412176697 (0.05555743443724691) with: {'dropout_rate': 0.0, 'learn_rate': 0.01}
0.7551020333657459 (0.05493466721806976) with: {'dropout_rate': 0.0, 'learn_rate': 0.1}
0.7806122347104306 (0.04086849117629243) with: {'dropout_rate': 0.1, 'learn_rate': 0.001}
0.7806122377514839 (0.03587767332968649) with: {'dropout_rate': 0.1, 'learn_rate': 0.01}
0.6658163286593496 (0.04451840561417468) with: {'dropout_rate': 0.1, 'learn_rate': 0.1}
0.7882652916774457 (0.04183757813687412) with: {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.7627551072106069 (0.042490838213417065) with: {'dropout_rate': 0.2, 'learn_rate': 0.01}
0.6836734623933325 (0.011916249369115174) with: {'dropout_rate': 0.2, 'learn_rate': 0.1}


In [18]:
# Do a grid search to optimize kernel initialization and activation functions
# import necessary packages

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(activation, init):
    # create model
    model = Sequential()
    model.add(Dense(40, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(20, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
activation = ['softmax', 'relu', 'tanh', 'linear']
init = ['uniform', 'normal', 'zero']

# make a dictionary of the grid search parameters
param_grid = dict(activation = activation, init = init)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .... activation=softmax, init=uniform, score=0.725, total=   3.2s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s remaining:    0.0s


[CV] .... activation=softmax, init=uniform, score=0.748, total=   2.7s
[CV] activation=softmax, init=uniform ................................


...

[CV] ....... activation=softmax, init=zero, score=0.700, total=   3.4s
[CV] activation=relu, init=uniform ...................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   25.2s remaining:    0.0s


[CV] ....... activation=relu, init=uniform, score=0.733, total=   2.4s
[CV] activation=relu, init=uniform ...................................
[CV] ....... activation=relu, init=uniform, score=0.763, total=   2.5s
[CV] activation=relu, init=uniform ...................................
[CV] ....... activation=relu, init=uniform, score=0.846, total=   2.5s
[CV] activation=relu, init=normal ....................................
[CV] ........ activation=relu, init=normal, score=0.740, total=   3.1s
[CV] activation=relu, init=normal ....................................
[CV] ........ activation=relu, init=normal, score=0.763, total=   2.3s
[CV] activation=relu, init=normal ....................................
[CV] ........ activation=relu, init=normal, score=0.823, total=   2.8s
[CV] activation=relu, init=zero ......................................
[CV] .......... activation=relu, init=zero, score=0.611, total=   2.7s
[CV] activation=relu, init=zero ......................................
[CV] .

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  1.8min finished


Best: 0.7959183487965136, using {'activation': 'tanh', 'init': 'normal'}
0.7653061007054485 (0.041756734489659664) with: {'activation': 'softmax', 'init': 'uniform'}
0.7729591775913628 (0.04620979028583469) with: {'activation': 'softmax', 'init': 'normal'}
0.6683673420730902 (0.04092231924913523) with: {'activation': 'softmax', 'init': 'zero'}
0.7806122514362238 (0.04782512280485408) with: {'activation': 'relu', 'init': 'uniform'}
0.7755101941982094 (0.03478923012909107) with: {'activation': 'relu', 'init': 'normal'}
0.6683673420730902 (0.04092231924913523) with: {'activation': 'relu', 'init': 'zero'}
0.7857142844978644 (0.03173683426123371) with: {'activation': 'tanh', 'init': 'uniform'}
0.7959183487965136 (0.04092362400694938) with: {'activation': 'tanh', 'init': 'normal'}
0.6683673420730902 (0.04092231924913523) with: {'activation': 'tanh', 'init': 'zero'}
0.7933673384238262 (0.037313666021535755) with: {'activation': 'linear', 'init': 'uniform'}
0.788265297911605 (0.030102057826861

In [19]:
# Do a grid search to find the optimal number of neurons in each hidden layer
# import necessary packages

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(neuron1, neuron2):
    # create model
    model = Sequential()
    model.add(Dense(neuron1, input_dim = 8, kernel_initializer= 'uniform', activation= 'linear'))
    model.add(Dense(neuron2, input_dim = neuron1, kernel_initializer= 'uniform', activation= 'linear'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
neuron1 = [40, 60, 80]
neuron2 = [20, 40, 60]

# make a dictionary of the grid search parameters
param_grid = dict(neuron1 = neuron1, neuron2 = neuron2)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), refit = True, verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] neuron1=40, neuron2=20 ..........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .............. neuron1=40, neuron2=20, score=0.771, total=   2.4s
[CV] neuron1=40, neuron2=20 ..........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s remaining:    0.0s


[CV] .............. neuron1=40, neuron2=20, score=0.763, total=   2.5s
[CV] neuron1=40, neuron2=20 ..........................................


...

[CV] .............. neuron1=40, neuron2=60, score=0.838, total=   2.6s
[CV] neuron1=60, neuron2=20 ..........................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   27.4s remaining:    0.0s


[CV] .............. neuron1=60, neuron2=20, score=0.756, total=   2.5s
[CV] neuron1=60, neuron2=20 ..........................................
[CV] .............. neuron1=60, neuron2=20, score=0.771, total=   3.4s
[CV] neuron1=60, neuron2=20 ..........................................
[CV] .............. neuron1=60, neuron2=20, score=0.838, total=   4.5s
[CV] neuron1=60, neuron2=40 ..........................................
[CV] .............. neuron1=60, neuron2=40, score=0.763, total=   2.7s
[CV] neuron1=60, neuron2=40 ..........................................
[CV] .............. neuron1=60, neuron2=40, score=0.771, total=   3.3s
[CV] neuron1=60, neuron2=40 ..........................................
[CV] .............. neuron1=60, neuron2=40, score=0.838, total=   3.6s
[CV] neuron1=60, neuron2=60 ..........................................
[CV] .............. neuron1=60, neuron2=60, score=0.756, total=   3.2s
[CV] neuron1=60, neuron2=60 ..........................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  1.4min finished


Best: 0.7908163113253457, using {'neuron1': 40, 'neuron2': 40}
0.7908163082842924 (0.03370616130132414) with: {'neuron1': 40, 'neuron2': 20}
0.7908163113253457 (0.028142952635149494) with: {'neuron1': 40, 'neuron2': 40}
0.7908163082842924 (0.03370616130132414) with: {'neuron1': 40, 'neuron2': 60}
0.7882652948705517 (0.035904875148556774) with: {'neuron1': 60, 'neuron2': 20}
0.7908163082842924 (0.03370616130132414) with: {'neuron1': 60, 'neuron2': 40}
0.7831632710841238 (0.03367863917165869) with: {'neuron1': 60, 'neuron2': 60}
0.7882652948705517 (0.0353583568133872) with: {'neuron1': 80, 'neuron2': 20}
0.7857142844978644 (0.03173683426123371) with: {'neuron1': 80, 'neuron2': 40}
0.7857142844978644 (0.03173683426123371) with: {'neuron1': 80, 'neuron2': 60}


In [20]:
# generate predictions with optimal hyperparameters
y_pred = grid.predict(X_standardized)

In [21]:
print(y_pred.shape)

(392, 1)


In [22]:
print(y_pred[:5])

[[0]
 [1]
 [0]
 [1]
 [1]]


In [23]:
# Generate a classification report
from sklearn.metrics import classification_report, accuracy_score

print(accuracy_score(Y, y_pred))
print(classification_report(Y, y_pred))

0.7857142857142857
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       262
           1       0.72      0.58      0.64       130

    accuracy                           0.79       392
   macro avg       0.76      0.74      0.75       392
weighted avg       0.78      0.79      0.78       392

